## The Traveling Salesman Problem (TSP)

### Problem

The TSP can be defined as follows: for a given list of cities and the distances between each pair of them, we want to find the shortest possible route that goes to each city once and returns to the origin city.

There is a class of Traveling Salesman Problems that assumes that the distance of going from city $i$ to city $j$  is the same as going form city $j$ to city $i$, this type of Traveling Salesman Problem  is also known as the symmetric Traveling Salesman Problem. 
In this example, we use Euclidean distances, but the TSP model formulation is valid independent of the way in which the individual distances are determined.

### Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. 
The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation of the TSP that identifies the shortest route that goes to all the cities once and returns to the origin city.

### Sets

$N = \{1,2,\ldots,n \}$: set onf cities

$\text{E}= \{(i,j) \in N \times N \}$: set of allowed pairings

$S \subset N$: a subset of the set cities.

$G = (N, E)$: a graph where the set $N$ defines the set of nodes and the set $E$ defines the set of edges. 

### Parameters 

$c_{i, j} \in \mathbb{R}_+$: distance from city $i$ to city $j$, for all $(i, j) \in E$. 

The distance from city $i$ to city $j$ is the same as the distance from city $j$ to city $i$, i.e. $c_{i,j} = c_{j,i}$. For this reason, this TSP is also called the symmetric Traveling Salesman Problem.

###  Decision variables

$x_{i, j} \in \{0, 1\}$: this variable is equal to 1, if we decide to connect city $i$ with city $j$. Otherwise, the decision variable is equal to zero.

### Objective function

- **Shortest Route**: 
Minimize the total distance of a route. 
A route is a sequence of cities where the salesperson visits each city only once and returns to the starting capital city.

\begin{equation}
\text{min} \quad z = \sum_{(i,j) \in \text{E}} c_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints 

- **Symmetry constraints**:
For each edge $(i,j)$, ensure that the city $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in E
\tag{1}
\end{equation}

- **Entering and leaving a city**:
For each city $i$, ensure that this city is connected to two other cities. 

\begin{equation}
\sum_{(i,j) \in \text{E}} x_{i,j} = 2, \quad \forall i \in N
\tag{2}
\end{equation}

- **Subtour elimination**:
These constraints ensure that for any subset of cities $S$ of the set of $N$, there is no cycle. 
That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S} x_{i,j} \leq |S|-1, \quad \forall S \subset N, \quad S \not= \emptyset
\tag{3}
\end{equation}

- **Remark**: 
In general, if the number of cities of the TSP is $n$, then the possible number of routes is n\!.
Since there are an exponential number of constraints ($2^{n} - 2$) to eliminate cycles, we use lazy constraints to dynamically eliminate those cycles. 

### Lazy Constraints

The basic idea of **Lazy Constraints** is to initially formulate the problem only with the most essential constraints, omitting those that are only rarely violated. 

These other constraints are checked and added one-by-one to the model only if the current solution violates any of them. 

In other words, some constraints are generated in a lazy fashion, i.e. the constraint is added to the model only if the solution violates it.

Lazy constraints have to be enabled by setting model parameter ```lazyConstraints``` to 1.

```
model = gp.Model()
model.Params.lazyConstraints = 1
```

To define a callback, create a function that accepts two arguments: ```model``` and ```where```. 
Thecallback is then passed to model when calling ```optimize()```.

```
def my_callback(model, where):
    # Callback is called when some event occur. The type of event is
    # distinguished using argument ’’where’’.
    # In this case, we want to perform something when an integer
    # solution is found, which corresponds to ’’GRB.Callback.MIPSOL’’.
    if where == GRB.Callback.MIPSOL:
        # TODO : your code here ...
        # Get the value of variable x[i,j] from the solution .
        # You may also pass a list of variables to the method .
        value = model.cbGetSolution(x[i,j])
        # Add lazy constraint to model.
        model.cbLazy(...)
        
model.optimize(my_callback)
```

### Problem

Consider a salesperson that needs to visit customers at each state capital of the continental US. 

The salesperson wants to identify the shortest route that goes to all the state capitals.

### Data

The capital names and coordinates are read from a json file.

JSON: stands for JavaScript Object Notation, is a text format for storing and transporting data, is "self-describing" and easy to understand.

In [31]:
# library to create maps
import json

# Read capital names and coordinates from json file
capitals_json = json.load(open('instances/capitals.json'))
capitals = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']:
      capital = capitals_json[state]['capital']
      capitals.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

### Data computation

The following function calculates the distance for each pair of state capitals. 

Since we are solving the _symmetric_ traveling salesman problem, we use _combinations_ of cities.

In [32]:
import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(x, y):
    c1 = coordinates[x]
    c2 = coordinates[y]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

### Model

We now write the model for the TSP, by defining decision variables, constraints, and objective function. 

Because this is the _symmetric_ traveling salesman problem, we can make it more efficient by setting the _object_ x[j,i] to x[i,j], instead of a constraint.

In [33]:
import gurobipy as gp

model = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = model.addVars(dist.keys(), obj=dist, vtype=gp.GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j,i] = vars[i,j]  # edge in opposite direction

# Constraints: two edges incident to each city
cons = model.addConstrs(vars.sum(e, '*') == 2 for e in capitals)

### Callback Definition

Subtour constraints prevent multiple loops in a TSP tour. 

Because there are an exponential number of these constraints, we don't want to add them all to the model. 

Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [34]:
# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
            
    return cycle

In [35]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == gp.GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour)-1)

### Solve the model

In [36]:
model._vars = vars
model.Params.lazyConstraints = 1
model.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 1128 columns and 2256 nonzeros
Model fingerprint: 0x63641a38
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 48 rows, 1128 columns, 2256 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 1.611858e+02, 72 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  161.18576    0   12          -  161.1857

### Analysis

We retrieve the optimal solution of the TSP and verify that the optimal route (or tour) goes to all the cities and returns to the origin city.

In [37]:
# Retrieve solution

vals = model.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

The optimal route is displayed in the following map.

In [38]:
# Map the solution

# library to create maps
import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

### Conclusions

The Traveling Salesman Problem (TSP) is the most popular combinatorial optimization problem. 

This problem is very easy to explain, although it is very complicated to solve. 

The largest TSP problem solved has 85,900 cities. 

The TSP is a source of discovery for new approaches to solve complex combinatorial optimization problems and has led to many applications.

In this modeling example, we have shown how to formulate the symmetric Traveling Salesman Problem as a MIP problem. 

We also showed how to dynamically eliminate subtours by using lazy constraints. 

### References

[1] D. L. Applegate, R. E. Bixby, V. Chvatal and W. J. Cook , The Traveling Salesman Problem: A Computational Study, Princeton University Press, Princeton, 2006.

[2] http://www.math.uwaterloo.ca/tsp/index.html

[3] https://www.youtube.com/watch?v=q8nQTNvCrjE&t=35s

[4] http://www.math.uwaterloo.ca/tsp/concorde.html

[5] https://github.com/Gurobi/modeling-examples/tree/master/traveling_salesman